# LEARNING PYTORCH WITH EXAMPLES

- 원본 : [링크](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)
- Author : [Justin Johnson](https://github.com/jcjohnson/pytorch-examples)
- 번역 & 주석 추가 : [박범진](https://github.com/pbj0812)

## PyTorch: Defining new autograd functions

각각의 원시 autograd 연산자는 Tensor들에 대해서 작동하는 함수들이다.<br>
forward 함수는 입력 Tensor를 이용하여 출력 Tensor를 계산한다.<br>
backward 함수는 Tensor에 대한 gradient를 받는다.
<br>
<br>
Pytorch에서 torch.autograd.Function의 정의 및 forward와 backward 함수의 구현을 통해<br> 
쉽게 autograd operator를 정의할 수 있다. 우리는 그것을 불러오고 인스턴스를 생성하고 <br>
입력 데이터가 포함된 Tensor를 전달함으로써 새로운 autograd operator를 사용할 수 있다.
<br>
<br>
이 예제에서 우리는 비선형의 ReLU를 만들기 위한 우리의 커스텀 autograd를 정의하고,<br> 
two-layer 네트워크를 구현하는데에 사용한다.

In [1]:
# -*- coding: utf-8 -*-
import torch

In [2]:
class MyReLU(torch.autograd.Function):
    """
    우리는 torch.autograd.Function의 하위분류 및 Tensor에서 작동되는 
    forward와 backward의 구현을 통해 우리의 커스텀 autograd 함수를 만들 수 있다.
    """
    
    @staticmethod
    def forward(ctx, input):
        """
        forward에서 우리는 입력값과 출력값을 가지는 Tensor를 얻는다. 
        ctx는 backward 계산을 위한 은닉 정보로 사용되는 객체이다.
        ctx.save_for_backward 방법을 사용해서 backward 계산을 위한
        임의의 객체를 숨길 수 있다.
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        backward 과정에서 우리는 출력값에 관한 loss의 gradient를 포함하는 Tensor를 얻을 수 있다, 
        그리고 우리는 입력값에 대한 loss의 gradient를 계산 할 필요가 있다.
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        
        return grad_input

In [3]:
dtype = torch.float
#device = torch.device("cpu")
device = torch.device("cuda:0")

In [4]:
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력값과 출력값을 가지는 random Tensor 생성
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# random wights를 가지는 Tensor 생성
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6

In [5]:
since = time.time()
for t in range(10000):
    # 우리의 함수를 적용하기 위해서는, Function.apply 메소드를 사용한다. 우리는 이것을 'relu'라고 부른다.
    relu = MyReLU.apply
    
    # forward : 계산을 통한 y 예측
    # 우리의 custom autograd operation을 사용하여 ReLU를 계산한다.
    y_pred = relu(x.mm(w1)).mm(w2)
    
    # loss 계산
    loss = (y_pred - y).pow(2).sum()
    
    if t%100 == 0:
        print(t, loss.item())
    
    # backward를 위한 autograd 계산
    loss.backward()
    
    # gradient descent를 통한 weight 갱신
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # gradient 초기화
        w1.grad.zero_()
        w2.grad.zero_()
        
now = time.time() - since

0 28359880.0
100 318.4908142089844
200 1.4441989660263062
300 0.01257424708455801
400 0.0003245252009946853
500 5.221130413701758e-05
600 2.0720131942653097e-05
700 1.1750641533581074e-05
800 7.879493750806432e-06
900 5.752538527303841e-06
1000 4.511611678026384e-06
1100 3.6444782836042577e-06
1200 3.0529231480613817e-06
1300 2.560029997766833e-06
1400 2.2545702904608333e-06
1500 1.947849114003475e-06
1600 1.705643740024243e-06
1700 1.5552759577985853e-06
1800 1.4023765970705426e-06
1900 1.2828525086661102e-06
2000 1.1786255527113099e-06
2100 1.0955524203382083e-06
2200 9.986133591155522e-07
2300 9.239444125341834e-07
2400 8.66971276991535e-07
2500 8.2318501881673e-07
2600 7.620055271218007e-07
2700 7.427700552398164e-07
2800 6.898133051436162e-07
2900 6.461937118729111e-07
3000 6.094330728956265e-07
3100 5.715688757845783e-07
3200 5.697712595065241e-07
3300 5.208008246881946e-07
3400 5.071437954029534e-07
3500 4.785377427651838e-07
3600 4.6707421574865293e-07
3700 4.4087587980357057e-

In [6]:
now

30.283974170684814